In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
path = "/content/drive/MyDrive/Dataset/Folds"
def load_data(dataset, fold):
  uri_val = "{}/{}/{}/prob_train.csv".format(path, dataset, fold)
  uri_test = "{}/{}/{}/prob_test.csv".format(path, dataset, fold)
  uri_test_pred = "{}/{}/{}/pred_test.csv".format(path, dataset, fold)
  df_val   = pd.read_table(uri_val, sep=',')
  df_test = pd.read_table(uri_test, sep=',')
  df_test_pred = pd.read_table(uri_test_pred, sep=',')
  val, class_val = df_val.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'class']), df_val['class']
  test, class_test = df_test_pred.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'class']), df_test_pred["class"]  
  return val, class_val, test, class_test


In [4]:
machines = ["SVM", "LR", "RF", "NB", "EXTRA", "KNN", "MLP", "CNN"]
features = ["CV", "TFIDF", "W2V", "GLOVE", "FAST"]
labels = ["0", "1", "2"]
clfs = {}

for machine in machines:
  clfs[machine] = []
  for feature in features:
    if machine == "CNN" and feature == "TFIDF":
      feature = "TF"
    for label in labels:
      clfs[machine].append("{}-{}-{}".format(machine, feature, label))


clfs_by_features = {}
for feature in features:
  clfs_by_features[feature] = []
  for machine in machines:
    for label in labels:
      if machine == "CNN" and feature == "TFIDF":
        clfs_by_features[feature].append("CNN-TF-{}".format(label))
      else:
        clfs_by_features[feature].append("{}-{}-{}".format(machine, feature, label))

In [5]:
folds = ["F1", "F2", "F3", "F4", "F5"]
pred = {
    "F1": {},
    "F2": {},
    "F3": {},
    "F4": {},
    "F5": {},
}
export = {}
for fold in folds:
  val, class_val, test, class_test = load_data("td", fold)  
  export.update({fold: {}})
  print("=============== {} ===============".format(fold))
  for machine, clf in clfs.items():
    cols = clfs[machine]
    val_vision = val[cols]    
    test_vision = test[cols]
    
    # Instancia de MLP
    lr = LogisticRegression(random_state=42,  max_iter=500)

    lr.fit(val_vision, class_val)
    y_pred = lr.predict(test_vision)
    f1 = f1_score(class_test, y_pred, average="macro")
    pred[fold].update({machine: f1})
    export[fold].update({machine: "%.3f" % f1})
    print("{}: ".format(machine), f1)

=============== F1 ===============
SVM:  0.7296244716455611
LR:  0.7290244091214896
RF:  0.6485832289927996
NB:  0.6522998660032365
EXTRA:  0.6186032489858709
KNN:  0.6534997557628647
MLP:  0.7093871316511203
CNN:  0.7245991207894513
=============== F2 ===============
SVM:  0.7262150696023575
LR:  0.7115050257709115
RF:  0.6170293548345253
NB:  0.6537955473185327
EXTRA:  0.6010850814662466
KNN:  0.6527217906252477
MLP:  0.6930807879343851
CNN:  0.7349545062178487
=============== F3 ===============
SVM:  0.7344175999510707
LR:  0.7318882053826407
RF:  0.6357890909119894
NB:  0.6553635316863841
EXTRA:  0.6137210875695344
KNN:  0.664209499789965
MLP:  0.7032122197638152
CNN:  0.7303482657404627
=============== F4 ===============
SVM:  0.7319752742495874
LR:  0.726209957433479
RF:  0.6358754193922116
NB:  0.6532464356547852
EXTRA:  0.6048822189573566
KNN:  0.6626319861050958
MLP:  0.695852501180022
CNN:  0.717082880866193
=============== F5 ===============
SVM:  0.7049535022259864
LR:  0.7

In [6]:
df = pd.DataFrame(data=export)
df.to_excel("machine.xlsx")

In [7]:
scores = {}
for machine in machines:
  scores.update({machine: []})
for fold, items in pred.items():
  for clf, score in items.items():
    scores[clf].append(score)

export = {}
for clf, score in scores.items():
  print("========= ", clf, "===========")
  print("Mean: ", np.mean(score))
  print("STD: ", np.std(score))
  export.update({clf: {
      "Mean": "%.3f" % np.mean(score),
      "STD": "%.3f" % np.std(score)
  }})


=========  SVM ===========
Mean:  0.7254371835349126
STD:  0.010593528190570642
=========  LR ===========
Mean:  0.7251623945477075
STD:  0.007097340388220048
=========  RF ===========
Mean:  0.6349146434243849
STD:  0.010139753184135307
=========  NB ===========
Mean:  0.6526891145319729
STD:  0.0022105569916129357
=========  EXTRA ===========
Mean:  0.6108923769460844
STD:  0.006747149254629535
=========  KNN ===========
Mean:  0.6578352878547153
STD:  0.00472331362597924
=========  MLP ===========
Mean:  0.6979045992643005
STD:  0.007560315751345792
=========  CNN ===========
Mean:  0.7230007168752185
STD:  0.009580320901296308


In [8]:
df = pd.DataFrame(data=export)
df.to_excel("mean-std-machine.xlsx")

# Grupo B

In [9]:
pred = {
    "F1": {},
    "F2": {},
    "F3": {},
    "F4": {},
    "F5": {},
}

In [10]:
folds = ["F1", "F2", "F3", "F4", "F5"]

export = {}
for fold in folds:
  export.update({fold: {}})
  val, class_val, test, class_test = load_data("union", fold)  
  print("=============== {} ===============".format(fold))
  for feature, clfs in clfs_by_features.items():
    # for clf in clfs:
    cols = clfs
    # # Instancia de MLP
    val_vision = val[cols]    
    test_vision = test[cols]

    lr = LogisticRegression(random_state=42,  max_iter=500)
    lr.fit(val_vision, class_val)
    y_pred = lr.predict(test_vision)
    f1 = f1_score(class_test, y_pred, average="macro")
    pred[fold].update({feature: f1})
    export[fold].update({feature: "%.3f" % f1})
    print(feature)
    print("Accuracy: ",accuracy_score(class_test, y_pred))
    print("F1: ", f1)
    # print("")


=============== F1 ===============
CV
Accuracy:  0.9016094242575079
F1:  0.8701599764610789
TFIDF
Accuracy:  0.9029367844698856
F1:  0.8705565147174653
W2V
Accuracy:  0.8599634975941596
F1:  0.7986524025876042
GLOVE
Accuracy:  0.8486809357889498
F1:  0.7638402956673825
FAST
Accuracy:  0.8405508544881367
F1:  0.7573319283910305
=============== F2 ===============
CV
Accuracy:  0.8949551941586459
F1:  0.8596425048386731
TFIDF
Accuracy:  0.8979422502489214
F1:  0.8624545589890031
W2V
Accuracy:  0.8549618320610687
F1:  0.7878722665168465
GLOVE
Accuracy:  0.8494855625622303
F1:  0.764277652437047
FAST
Accuracy:  0.8420179223365416
F1:  0.7557098159993476
=============== F3 ===============
CV
Accuracy:  0.8972784600066379
F1:  0.8642274525658472
TFIDF
Accuracy:  0.9029206770660472
F1:  0.8709364284787969
W2V
Accuracy:  0.861599734483903
F1:  0.797679540058028
GLOVE
Accuracy:  0.8571191503484898
F1:  0.780783357656158
FAST
Accuracy:  0.8340524394291404
F1:  0.7370773760286485
=============== F

In [11]:
df = pd.DataFrame(data=export)
df.to_excel("feature.xlsx")

In [12]:
# print(pred)
scores = {}
for feature in features:
  scores.update({feature: []})
for fold, items in pred.items():
  for feature, score in items.items():
    scores[feature].append(score)


export = {}
for feature, score in scores.items():
  print("========= ", feature, "===========")
  print("Mean: ", np.mean(score))
  print("STD: ", np.std(score))
  export.update({feature: {
      "Mean": "%.3f" % np.mean(score),
      "STD": "%.3f" % np.std(score)
  }})

=========  CV ===========
Mean:  0.8662496298607388
STD:  0.00386572156110815
=========  TFIDF ===========
Mean:  0.8702129468092064
STD:  0.004295443807453683
=========  W2V ===========
Mean:  0.7930830733486249
STD:  0.00832163749498815
=========  GLOVE ===========
Mean:  0.7734257860918967
STD:  0.0076893355884087855
=========  FAST ===========
Mean:  0.75305649562121
STD:  0.008293474757476271


In [13]:
df = pd.DataFrame(data=export)
df.to_excel("mean-std-feature.xlsx")

# Grupo C

In [14]:
pred = []
export = {}

In [15]:
folds = ["F1", "F2", "F3", "F4", "F5"]

for fold in folds:
  # export.update({fold: {}})
  val, class_val, test, class_test = load_data("td", fold)  
  print("=============== {} ===============".format(fold))
  val_vision = val
  test_vision = test

  lr = LogisticRegression(random_state=42,  max_iter=500)
  lr.fit(val_vision, class_val)
  y_pred = lr.predict(test_vision)
  f1 = f1_score(class_test, y_pred, average="macro")
  pred.append(f1)
  export[fold] = "%.3f" % f1
  print(" TODOS ")
  print("Accuracy: ",accuracy_score(class_test, y_pred))
  print("F1: ", f1)
  # print("")

=============== F1 ===============


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


 TODOS 
Accuracy:  0.9013516239661086
F1:  0.6767032782517434
=============== F2 ===============
 TODOS 
Accuracy:  0.8894492636675408
F1:  0.6509460887239514
=============== F3 ===============
 TODOS 
Accuracy:  0.898325600161388
F1:  0.6684922030956552
=============== F4 ===============


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


 TODOS 
Accuracy:  0.8904358353510896
F1:  0.6417462050130068
=============== F5 ===============
 TODOS 
Accuracy:  0.8847861178369653
F1:  0.6379230292255741


In [16]:
df = pd.DataFrame(data=export, index=[0])
df.to_excel("all.xlsx")

print("========= ", "TODOS", "===========")
print("Mean: ", np.mean(pred))
print("STD: ", np.std(pred))

=========  TODOS ===========
Mean:  0.6551621608619862
STD:  0.015077536665802664


# CLUSTERS

In [17]:
def load_clfs(fold):  
  import random
  clfs_fold = []
  quantidade = random.randint(5, len(fold)-1)
  sorteados = 0
  indices = []
  while sorteados < quantidade:
    item = random.randint(0, len(fold)-1)
    if not item in indices:
      indices.append(item)
      sorteados += 1

  for i in indices:
    # print(i)
    # print(fold[i])
    clfs_fold.append(random.choices(fold[i].split(", "))[0])
  return quantidade, clfs_fold

F1 = [
    "RF-CV, RF-TFIDF",
    "SVM-GLOVE, MLP-GLOVE",
    "NB-CV, NB-TFIDF",
    "LR-W2V, SVM-W2V, MLP-W2V, MLP-FAST",
    "NB-GLOVE, LR-GLOVE, RF-GLOVE, EXTRA-GLOVE, KNN-GLOVE",
    "EXTRA-W2V, KNN-W2V, RF-W2V",
    "KNN-FAST, EXTRA-FAST, RF-FAST",
    "CNN-FAST, CNN-W2V, CNN-GLOVE",
    "EXTRA-CV, EXTRA-TFIDF",
    "NB-FAST, NB-W2V, KNN-TFIDF",
    "SVM-TFIDF, MLP-TFIDF, LR-CV, LR-TFIDF, SVM-CV, CNN-CV, CNN-TF, KNN-CV, MLP-CV",
    "SVM-FAST, LR-FAST"
]

F2 = [
    "MLP-GLOVE, LR-GLOVE, SVM-GLOVE",
    "KNN-TFIDF, NB-W2V, NB-FAST",
    "LR-W2V, SVM-W2V, MLP-W2V, MLP-FAST",
    "RF-FAST, KNN-FAST, EXTRA-FAST",
    "LR-FAST, SVM-FAST",
    "SVM-CV, LR-CV, SVM-TFIDF, MLP-TFIDF",
    "NB-CV, NB-TFIDF",
    "KNN-W2V, EXTRA-W2V, RF-W2V, CNN-FAST, CNN-W2V, CNN-GLOVE, KNN-CV",
    "EXTRA-GLOVE, RF-GLOVE, KNN-GLOVE, NB-GLOVE",
    "EXTRA-TFIDF, RF-TFIDF, RF-CV, EXTRA-CV",
    "MLP-CV, MLP-TFIDF",
    "CNN-CV, CNN-TF"
]

F3 = [
    "EXTRA-W2V, RF-W2V",
    "RF-GLOVE, EXTRA-GLOVE, KNN-GLOVE",
    "MLP-W2V, SVM-W2V, LR-W2V",
    "KNN-W2V, RF-FAST, EXTRA-FAST, KNN-FAST",
    "MLP-GLOVE, SVM-GLOVE, LR-GLOVE, NB-GLOVE",
    "CNN-GLOVE, CNN-W2V",
    "KNN-TFIDF, NB-W2V, NB-FAST",
    "NB-CV, NB-TFIDF",
    "LR-TFIDF, LR-CV, MLP-CV, MLP-TFIDF, SVM-CV, SVM-TFIDF",
    "CNN-TF, CNN-CV, EXTRA-TFIDF, EXTRA-CV, RF-TFIDF, RF-CV, CNN-FAST, KNN-CV",
    "LR-FAST, SVM-FAST, MLP-FAST"
]

F4 = [
    "EXTRA-W2V, RF-W2V, KNN-W2V",
    "KNN-TFIDF, NB-W2V, NB-FAST",
    "NB-TFIDF, NB-CV",
    "MLP-FAST, MLP-W2V",
    "NB-GLOVE, RF-GLOVE, CNN-GLOVE, EXTRA-GLOVE",
    "KNN-FAST, EXTRA-FAST, RF-FAST",
    "SVM-W2V, LR-W2V",
    "LR-CV, LR-TFIDF",
    "CNN-GLOVE, CNN-CV, CNN-W2V",
    "MLP-GLOVE, LR-GLOVE, SVM-GLOVE",
    "SVM-FAST, LR-FAST",
    "EXTRA-CV, RF-CV",
    "SVM-CV, SVM-TFIDF, MLP-CV, MLP-TFIDF",
    "CNN-TF, CNN-CV, CNN-FAST, RF-TFIDF, EXTRA-TFIDF"
]

F5 = [
    "RF-FAST, EXTRA-FAST, KNN-FAST",
    "EXTRA-W2V, RF-W2V, KNN-W2V",
    "LR-W2V, SVM-W2V",
    "MLP-W2V, MLP-FAST",
    "NB-GLOVE, LR-GLOVE, SVM-GLOVE, MLP-GLOVE, EXTRA-GLOVE, RF-GLOVE, KNN-GLOVE",
    "KNN-TFIDF, NB-W2V, NB-FAST",
    "EXTRA-CV, RF-CV",
    "LR-FAST, SVM-FAST",
    "CNN-W2V, CNN-FAST",
    "MLP-CV, MLP-TFIDF, SVM-CV, SVM-TFIDF, LR-CV, LR-TFIDF, CNN-CV",
    "CNN-GLOVE, KNN-CV, NB-TFIDF, NB-CV",
    "RF-TFIDF, EXTRA-TFIDF, CNN-TF"
]

# Grupo D

In [18]:
clusters = {"F1": F1, "F2": F2, "F3": F3, "F4": F4, "F5": F5}
results = {}
for fold, cluster in clusters.items():
  cont = 1
  results.update({fold: {"clfs": [], "f1": 0}})
  while cont < 300:
    qt, clfs = load_clfs(cluster)
    pred = []
    # folds = ["F1"]
    # fold="F1"

    val, class_val, test, class_test = load_data("td", fold)
    cols = []
    # columns 
    for clf in clfs:
      for i in range(0,3):
        cols.append("{}-{}".format(clf, i))
    
    val_vision = val[cols]
    test_vision = test[cols]
    # Treinamento
    lr = LogisticRegression(random_state=42,  max_iter=500)
    lr.fit(val_vision, class_val)
    y_pred = lr.predict(test_vision)
    f1 = f1_score(class_test, y_pred, average="macro")
    pred.append(f1)
    # print("=============== {} ===============".format(fold))

    if f1 > results[fold]["f1"]:
      results[fold].update({"clfs": clfs, "f1": f1})
    
    cont += 1

In [19]:
results

{'F1': {'clfs': ['RF-CV',
   'MLP-GLOVE',
   'CNN-W2V',
   'LR-FAST',
   'MLP-W2V',
   'NB-TFIDF',
   'KNN-W2V'],
  'f1': 0.7348136817279608},
 'F2': {'clfs': ['NB-GLOVE',
   'MLP-CV',
   'MLP-FAST',
   'NB-CV',
   'MLP-GLOVE',
   'CNN-TF',
   'SVM-TFIDF'],
  'f1': 0.749859235708004},
 'F3': {'clfs': ['MLP-FAST',
   'CNN-FAST',
   'SVM-CV',
   'CNN-GLOVE',
   'KNN-FAST',
   'NB-FAST'],
  'f1': 0.7391761222985277},
 'F4': {'clfs': ['LR-CV',
   'CNN-GLOVE',
   'NB-GLOVE',
   'MLP-FAST',
   'SVM-FAST',
   'LR-GLOVE'],
  'f1': 0.7552656483083838},
 'F5': {'clfs': ['LR-FAST',
   'NB-FAST',
   'CNN-FAST',
   'LR-W2V',
   'SVM-GLOVE',
   'NB-TFIDF',
   'CNN-CV'],
  'f1': 0.720166639332382}}

In [20]:
df = pd.DataFrame(data=results)
df.to_excel("stacked.xlsx")

In [21]:
lista = []
for fold, result in results.items():
  lista.append(result["f1"])
print("========= ", "COMBINAÇÃO", "===========")
print("Mean: ", np.mean(lista))
print("STD: ", np.std(lista))

=========  COMBINAÇÃO ===========
Mean:  0.7398562654750517
STD:  0.012256264651276251
